In [1]:
alphabet = ['A','C','G','T']
score =[[0,4,2,4,8], \
        [4,0,4,2,8], \
        [2,4,0,4,8], \
        [4,2,4,0,8], \
        [8,8,8,8,8]]

In [5]:
score[-1][alphabet.index('C')]

8

In [9]:
score[alphabet.index('G')][-1]

8

# Programming Homework 3 Instructions 

We saw how to adapt dynamic programming to find approximate occurrences of a pattern in a text. Recall that:

- Rows of the dynamic programming matrix are labeled with bases from P and columns with bases from T
- Elements in the first row are set to 0
- Elements in the first column are set to 0, 1, 2, ..., as for edit distance
- Other elements are set in the same way as elements of a standard edit distance matrix
- The minimal value in the bottom row is the edit distance of the closest match between P and T
- First, download the provided excerpt of human chromosome 1

In [10]:
import wget
import os

url = 'https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta'
fname = os.path.basename(url)

if not os.path.isfile('data/'+fname):
    wget.download(url, out='data/')
else: 
    print(fname + " Already Exist! ")

chr1.GRCh38.excerpt.fasta Already Exist! 


Second, parse it using the readGenome function we wrote before.

Third, adapt the editDistance function we saw in practical (copied below) to answer questions 1 and 2 below. Your function should take arguments p (pattern), t (text) and should return the edit distance of the match between P and T with the fewest edits.

In [11]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [14]:
human_chr1 = readGenome('data/'+fname)

In [45]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]


In [46]:
pattern = 'GCGTATGC'
text = 'TATTGGCTATACGGTT'

editDistance(pattern,text)

11

In [47]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = 0 # Row == 0
        
    #print(D)
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return min(D[-1])

In [38]:
pattern = 'GCGTATGC'
text = 'TATTGGCTATACGGTT'

editDistance(pattern,text)

2

## Programming Homework 3

#### Question 1:  What is the edit distance of the best match between pattern `GCTGATCGATCGTACG` and the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [39]:
pattern = 'GCTGATCGATCGTACG'

editDistance(pattern,human_chr1)

3

#### Question 2:  What is the edit distance of the best match between pattern `GATTTACCAGATTGAG` and the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [40]:
pattern = 'GATTTACCAGATTGAG'
editDistance(pattern,human_chr1)

2

#### Question 3:
In a practical, we saw a function for finding the longest exact overlap (suffix/prefix match) between two strings. The function is copied below.

In [41]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

Say we are concerned only with overlaps that (a) are exact matches (no differences allowed), and (b) are at least `k` bases long. To make an overlap graph, we could call `overlap(a, b, min_length=k)` on every possible pair of reads from the dataset.  Unfortunately, that will be very slow!

Consider this: Say we are using `k=6`, and we have a read `a` whose length-6 suffix is `GTCCTA`.  Say `GTCCTA` does not occur in any other read in the dataset.  In other words, the 6-mer `GTCCTA` occurs at the end of read `a` and nowhere else.  It follows that `a`'s suffix cannot possibly overlap the prefix of any other read by 6 or more characters.

Put another way, if we want to find the overlaps involving a suffix of read `a` and a prefix of some other read, we can ignore any reads that don't contain the length-k suffix of `a`.  This is good news because it can save us a lot of work!

Here is a suggestion for how to implement this idea.  You don't have to do it this way, but this might help you.  Let every k-mer in the dataset have an associated Python `set` object, which starts out empty.  We use a Python dictionary to associate each k-mer with its corresponding \verb|set|set. (1) For every k-mer in a read, we add the read to the `set` object corresponding to that k-mer.  If our read is `GATTA` and `k=3`, we would add `GATTA` to the `set` objects for `GAT`, `ATT` and `TTA`.  We do this for every read so that, at the end, each `set` contains all reads containing the corresponding k-mer.  (2) Now, for each read \verb|a|a, we find all overlaps involving a suffix of `a`.  To do this, we take `a`'s length-k suffix, find all reads containing that k-mer (obtained from the corresponding `set`) and call `overlap(a, b, min_length=k)` for each.

The most important point is that we do not call `overlap(a, b, min_length=k)` if `b` does not contain the length-k suffix of `a`.

Download and parse the read sequences from the provided Phi-X FASTQ file. We'll just use their base sequences, so you can ignore read names and base qualities.  Also, no two reads in the FASTQ have the same sequence of bases.  This makes things simpler.

In [51]:
import wget
import os

url = 'https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq'
fname = os.path.basename(url)

if not os.path.isfile('data/'+fname):
    wget.download(url, out='data/')
    print(fname + " File Downloaded...")
else: 
    print(fname + " Already Exist! ")

ERR266411_1.for_asm.fastq Already Exist! 


Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match.  Ignore reverse complements.

- **Hint 1**: Your function should not take much more than 15 seconds to run on this 10,000-read dataset, and maybe much less than that.  (Our solution takes about 3 seconds.) If your function is much slower, there is a problem somewhere.
- **Hint 2**: Remember not to overlap a read with itself. If you do, your answers will be too high.
- **Hint 3**: You can test your implementation by making up small examples, then checking that (a) your implementation runs quickly, and (b) you get the same answer as if you had simply called `overlap(a, b, min_length=k)` on every pair of reads.  We also have provided a couple examples you can check against.

In [54]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0: # If line reaches end of file seq value become 0
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [57]:
a , b = 'ABCDEFG', 'XXXXX'
overlap(a, b, min_length=3)

0

In [55]:
phix_genome_fastq, _ = readFastq('data/'+fname)

In [92]:
# Brute Force 
def overlap_all_pairs(fastq,min_length=3):
    pairs = set()
    for a in fastq:
        for b in fastq:
            if a != b and overlap(a, b, min_length) > 0:
                pairs.add((a,b))
    return list(pairs)
                

In [93]:
from itertools import permutations
reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']
list(permutations(reads,2))

[('ABCDEFG', 'EFGHIJ'),
 ('ABCDEFG', 'HIJABC'),
 ('EFGHIJ', 'ABCDEFG'),
 ('EFGHIJ', 'HIJABC'),
 ('HIJABC', 'ABCDEFG'),
 ('HIJABC', 'EFGHIJ')]

In [103]:
def naive_overlap_all_pairs(reads,min_length=3):
    overlap_pairs = dict()
    for a,b in permutations(reads,2):
        overlap_length =  overlap(a, b, min_length) 
        if overlap_length > 0:
            overlap_pairs[(a,b)] = overlap_length
    return overlap_pairs.keys()

In [104]:
reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']
naive_overlap_all_pairs(reads, 3)

dict_keys([('ABCDEFG', 'EFGHIJ'), ('EFGHIJ', 'HIJABC'), ('HIJABC', 'ABCDEFG')])

In [100]:
overlap_all_pairs(reads, 4)

[]

In [88]:
reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']
overlap_all_pairs(reads, 4)

[('ACGTAC', 'GTACGT'),
 ('GTACGA', 'TACGAT'),
 ('ACGTAC', 'CGTACG'),
 ('CGTACG', 'TACGTA'),
 ('ACGTAC', 'GTACGA'),
 ('GTACGT', 'ACGTAC'),
 ('CGTACG', 'GTACGT'),
 ('CGTACG', 'TACGAT'),
 ('CGTACG', 'GTACGA'),
 ('TACGTA', 'ACGTAC'),
 ('TACGTA', 'CGTACG'),
 ('GTACGT', 'TACGTA')]

In [89]:
overlap_all_pairs(reads, 5)

[('GTACGA', 'TACGAT'),
 ('ACGTAC', 'CGTACG'),
 ('CGTACG', 'GTACGT'),
 ('CGTACG', 'GTACGA'),
 ('TACGTA', 'ACGTAC'),
 ('GTACGT', 'TACGTA')]

```
The Python defaultdict type behaves almost exactly like a regular Python dictionary, but if you try to access or modify a missing key, then defaultdict 
will automatically create the key and generate a default value for it. This makes defaultdict a valuable option for handling missing keys in dictionaries.
```

In [122]:
from collections import defaultdict

def fast_overlap_all_pairs(reads, k):
    dic = {}
    index = defaultdict(set)
    for read in reads:
        for i in range(len(read)-k+1):
            index[read[i:i+k]].add(read)
    #print(index)
    # make graph
    graph = defaultdict(set)
    for r in reads:
        for o in index[r[-k:]]:
            if r != o:
                if overlap(r, o, k):
                    graph[r].add(o)
    #print(graph)
    edges = 0
    for read in graph:
        edges += len(graph[read])
    return(edges, len(graph))

In [136]:
k = 3
reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']
dic = {}
index = defaultdict(set)
for read in reads:
    for i in range(len(read)-k+1):
        index[read[i:i+k]].add(read)
index

defaultdict(set,
            {'ABC': {'ABCDEFG', 'HIJABC'},
             'BCD': {'ABCDEFG'},
             'CDE': {'ABCDEFG'},
             'DEF': {'ABCDEFG'},
             'EFG': {'ABCDEFG', 'EFGHIJ'},
             'FGH': {'EFGHIJ'},
             'GHI': {'EFGHIJ'},
             'HIJ': {'EFGHIJ', 'HIJABC'},
             'IJA': {'HIJABC'},
             'JAB': {'HIJABC'}})

In [134]:
graph = defaultdict(set)
for read_a in reads:
    for read_b in index[r[-k:]]:
        if read_a != read_b:
            if overlap(read_a, read_b, k):
                graph[read_a].add(read_b)

graph

defaultdict(set, {'EFGHIJ': {'HIJABC'}, 'HIJABC': {'ABCDEFG'}})

In [135]:
edges = 0
for read in graph:
    edges += len(graph[read])
edges

2

In [119]:
reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']
fast_overlap_all_pairs(reads, 3)

defaultdict(<class 'set'>, {'ABCDEFG': {'EFGHIJ'}, 'EFGHIJ': {'HIJABC'}, 'HIJABC': {'ABCDEFG'}})


(3, 3)

In [144]:
from collections import defaultdict

def fast_overlap_all_pairs(reads, k):
    dic = {}
    index = defaultdict(set)
    for read in reads:
        for i in range(len(read)-k+1):
            index[read[i:i+k]].add(read)

    graph = defaultdict(set)
    for read_a in reads:
        for read_b in index[read_a[-k:]]:
            if read_a != read_b:
                if overlap(read_a, read_b, k):
                    graph[read_a].add(read_b)
                    
    #print(graph)
    edges = 0
    for read in graph:
        edges += len(graph[read])
    return(edges, len(graph))

In [145]:
phiX_pairs = fast_overlap_all_pairs(phix_genome_fastq, 30)
phiX_pairs

(904746, 7161)